### Imports necesarios

In [1]:
# import keras.losses
import numpy as np
import os
import config
import pickle
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential, losses, optimizers, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
# tf.config.set_visible_devices(tf.config.list_physical_devices('CPU')[0], 'CPU')
# tf.debugging.set_log_device_placement(True)

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


### Cargar datos de entrenamiento

In [3]:
with open(config.TRAINING_FILE, "rb") as f:
    df = pickle.load(f)

df.head()

,image_name,Male
20803,20803.jpg,male
20820,20820.jpg,male
24617,24617.jpg,male
29604,29604.jpg,male
11602,11602.jpg,male


In [4]:
im = plt.imread(f"{config.TRAINING_IMGS}20803.jpg")
img_width, img_height = im.shape
pixel_count = img_width * img_height

img_names = df['image_name'].values.reshape(1, df['image_name'].values.size)[0]
X_train = np.array([plt.imread(f"{config.TRAINING_IMGS}{img_name}") for img_name in img_names]) / 255
Y_train = np.where(df["Male"] == "male", 1, 0).T

In [5]:
# model = Sequential()
# model.add(Conv2D(32, 3, padding="valid", activation="relu",  input_shape=(img_width, img_height, 1)))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(32, 3, activation="relu"))
# model.add(MaxPooling2D((2, 2)))
# model.add(Flatten())
# model.add(Dense(32, activation="relu"))
# model.add(Dense(2, activation="sigmoid"))
#
# loss = losses.BinaryCrossentropy(from_logits=False)
# optimizer = optimizers.SGD(learning_rate=0.01)
# metrics = ["accuracy"]
#
# model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
#
# model.summary()



In [6]:
print("model.summary()")

model.summary()


In [7]:
def init_params():
    W1 = np.random.randn(2, pixel_count)
    b1 = np.random.randn(2, 1)
    W2 = np.random.randn(2, 2)
    b2 = np.random.randn(2, 1)
    return W1, b1, W2, b2

def ReLu(Z):
    return np.maximum(0, Z)

def softmax(Z):
    return np.exp(Z) / np.sum(np.exp(Z))

def forward_prop(X, W1, b1, W2, b2):
    Z1 = W1.dot(X) + b1
    A1 = ReLu(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def one_hot(Y):
    return np.vstack((np.array(np.where(Y == 1, 1, 0)), np.array(np.where(Y == 0, 1, 0))))

def deriv_ReLu(Z):
    return Z > 0

def back_prop(Z1, A1, A2, W2, X, Y):
    Y_one_hot = one_hot(Y)
    dZ2 = A2 - Y_one_hot
    dW2 = (1 / pixel_count) * dZ2.dot(A1.T)
    db2 = (1 / pixel_count) * np.sum(dZ2, axis=1).reshape(2, 1)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLu(Z1)
    dW1 = (1 / pixel_count) * dZ1.dot(X.T)
    db1 = (1 / pixel_count) * np.sum(dZ1, axis=1).reshape(2, 1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 -= alpha * dW1
    b1 -= alpha * db1
    W2 -= alpha * dW2
    b2 -= alpha * db2
    return W1, b1, W2, b2

def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()

    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(X, W1, b1, W2, b2)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2,  dW1, db1, dW2, db2, alpha)
        if i % 100 == 0:
            print(f"Iteration: {i}")
            print(f"Accuracy: {get_accuracy(get_predictions(A2), Y)}")

    print("Done")
    return W1, b1, W2, b2

In [8]:
def conv_block(input, n_filters):
    conv_maps = Conv2D(n_filters, kernel_size=(3, 3), activation=tf.nn.relu)(input)
    return MaxPooling2D(pool_size=(2, 2))(conv_maps)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation=tf.nn.relu,  input_shape=(img_width, img_height, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# model_input = tf.keras.Input(im.shape)
# n_filters_list = [32, 32, 64]
# dense_fully_connected_layers = [64, 32]
#
# conv_input = conv_output = model_input
# for n_filters in n_filters_list:
#     conv_output = conv_block(conv_input, n_filters)
#     conv_input = conv_output
#
# conv_output_flat = GlobalAveragePooling2D()(conv_output)
#
# dense_layer_input = dense_layer_output = conv_output_flat
# for dense_layer_size in dense_fully_connected_layers:
#     dense_layer_output = Dense(dense_layer_size, activation=tf.nn.relu)(dense_layer_input)
#     dense_layer_input = dense_layer_output
#
# output_layer = Dense(1, activation=tf.nn.sigmoid)(dense_layer_output)
# model = Model(model_input, output_layer, name="classification_model")



#
# conv_maps = Conv2D(32, 3, input_shape=(img_width, img_height, 1))
# activation_maps = Activation("relu")(conv_maps)
# # pooled_maps = MaxPooling2D(pool_size=(2,2))(activation_maps)
# #
# # tf.keras.Input(im.shape)
#
model.add(Conv2D(32, (3, 3), activation=tf.nn.relu))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
# #
model.add(Conv2D(64, (3, 3), activation=tf.nn.relu))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(.5))
# #
# model.add(Flatten(input_shape=(img_width, img_height)))
model.add(Dense(128, activation=tf.nn.relu))
model.add(Dense(2, activation=tf.nn.softmax))
# # model.add()
# # model = tf.keras.Sequential([
# #     Flatten(input_shape=(img_width, img_height)),
# #     Dense(128, activation=tf.nn.relu),
# #     Dense(2, activation=tf.nn.softmax)
# # ])
#
# loss = losses.CategoricalCrossentropy()
optimizer = optimizers.SGD(learning_rate=.01)
metrics = ["accuracy"]

model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=metrics)

Y_one_hot = one_hot(Y_train).T
print("n")
model.summary()
model.fit(X_train, Y_one_hot, verbose=1, batch_size=64, epochs=10)

n
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      320       
                                                                 
 batch_normalization (BatchN  (None, 254, 254, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      9248      
                                                                 
 batch_normalization_1 (Batc  (None, 125, 125, 32)     128       
 hNormalization)                                                 
                                                      

In [ ]:
model.summary()